In [142]:
from pymongo import MongoClient
import pandas as pd
from pandas.io.json import json_normalize

In [143]:

client = MongoClient("mongodb://localhost/companies")
db = client.get_database()

#  DF collection: companies

In [144]:

cursor = db["companies"].find({})
dfcompanies = json_normalize(cursor)

In [145]:
dfcompanies.columns

Index(['_id', 'name', 'permalink', 'crunchbase_url', 'homepage_url',
       'blog_url', 'blog_feed_url', 'twitter_username', 'category_code',
       'number_of_employees', 'founded_year', 'founded_month', 'founded_day',
       'deadpooled_year', 'tag_list', 'alias_list', 'email_address',
       'phone_number', 'description', 'created_at', 'updated_at', 'overview',
       'products', 'relationships', 'competitions', 'providerships',
       'total_money_raised', 'funding_rounds', 'investments', 'acquisitions',
       'offices', 'milestones', 'video_embeds', 'screenshots',
       'external_links', 'partners', 'image.available_sizes',
       'acquisition.price_amount', 'acquisition.price_currency_code',
       'acquisition.term_code', 'acquisition.source_url',
       'acquisition.source_description', 'acquisition.acquired_year',
       'acquisition.acquired_month', 'acquisition.acquired_day',
       'acquisition.acquiring_company.name',
       'acquisition.acquiring_company.permalink', 'ac

In [146]:
dfcompanies = dfcompanies[['name','category_code','total_money_raised', 'offices','founded_year']].explode('offices')

In [147]:

dfComp = dfcompanies[["offices"]].apply(lambda r: r.offices ,result_type="expand", axis=1)
dfcompoffi = pd.concat([dfcompanies,dfComp],axis=1)

In [148]:
dfcompoffi = dfcompoffi.drop(columns='offices')


In [154]:
dfcompoffi

,name,category_code,total_money_raised,founded_year,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,location
0,Wetpaint,web,$39.8M,2005.0,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253,"{'type': 'Point', 'coordinates': [-122.333253,..."
0,Wetpaint,web,$39.8M,2005.0,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431,"{'type': 'Point', 'coordinates': [-73.9964312,..."
1,Zoho,software,$0,2005.0,Headquarters,4900 Hopyard Rd,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945,"{'type': 'Point', 'coordinates': [-121.904945,..."
2,Omnidrive,network_hosting,$800k,2005.0,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,NaN,NaN,None
3,Flektor,games_video,$0,NaN,None,"8536 National Blvd, Suite A",None,90232,Culver City,CA,USA,34.025958,-118.379768,"{'type': 'Point', 'coordinates': [-118.379768,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18795,Willdan Group,other,$0,NaN,,2401 East Katella Avenue,Suite 300,92806-5909,Anaheim,CA,USA,33.806525,-117.882314,"{'type': 'Point', 'coordinates': [-117.882314,..."
18796,EnteGreat Solutions,software,$0,NaN,,,,,Birmingham,AL,USA,33.518885,-86.816068,"{'type': 'Point', 'coordinates': [-86.816068, ..."
18797,AfterLogic,software,$0,2002.0,Livingston,P.O. Box 2096,,07039,Livingston,NJ,USA,40.793024,-74.323554,"{'type': 'Point', 'coordinates': [-74.3235539,..."
18798,goBookmaker,web,$0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


In [150]:
import math
def asGeoJSON(lat,lng):
    try:
        lat = float(lat)
        lng = float(lng)
        if not math.isnan(lat) and not math.isnan(lng):
            return {
                "type":"Point",
                "coordinates":[lng,lat]
            }
    except Exception:
        print("Invalid data")
        return None
dfcompoffi["location"] = dfcompoffi[["latitude","longitude"]].apply(lambda x:asGeoJSON(x.latitude,x.longitude), axis=1)

In [155]:
dfcompoffi

,name,category_code,total_money_raised,founded_year,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,location
0,Wetpaint,web,$39.8M,2005.0,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253,"{'type': 'Point', 'coordinates': [-122.333253,..."
0,Wetpaint,web,$39.8M,2005.0,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431,"{'type': 'Point', 'coordinates': [-73.9964312,..."
1,Zoho,software,$0,2005.0,Headquarters,4900 Hopyard Rd,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945,"{'type': 'Point', 'coordinates': [-121.904945,..."
2,Omnidrive,network_hosting,$800k,2005.0,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,NaN,NaN,None
3,Flektor,games_video,$0,NaN,None,"8536 National Blvd, Suite A",None,90232,Culver City,CA,USA,34.025958,-118.379768,"{'type': 'Point', 'coordinates': [-118.379768,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18795,Willdan Group,other,$0,NaN,,2401 East Katella Avenue,Suite 300,92806-5909,Anaheim,CA,USA,33.806525,-117.882314,"{'type': 'Point', 'coordinates': [-117.882314,..."
18796,EnteGreat Solutions,software,$0,NaN,,,,,Birmingham,AL,USA,33.518885,-86.816068,"{'type': 'Point', 'coordinates': [-86.816068, ..."
18797,AfterLogic,software,$0,2002.0,Livingston,P.O. Box 2096,,07039,Livingston,NJ,USA,40.793024,-74.323554,"{'type': 'Point', 'coordinates': [-74.3235539,..."
18798,goBookmaker,web,$0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


### Pasamos el DF limpio a json para pasarlo a mongodb

In [156]:
dfcompoffi.to_json("cleaned_offices.json", orient="records")

# Descargamos y limpiamos el csv de Starbucks

In [139]:
StarBdf = pd.read_csv("StarbucksDirectory.csv")
StarBdf["location"] = StarBdf[["Latitude","Longitude"]].apply(lambda x:asGeoJSON(x.Latitude,x.Longitude), axis=1)
StarBdf.columns

Index(['Brand', 'Store Number', 'Store Name', 'Ownership Type',
       'Street Address', 'City', 'State/Province', 'Country', 'Postcode',
       'Phone Number', 'Timezone', 'Longitude', 'Latitude', 'location'],
      dtype='object')

In [140]:
StarBdf = StarBdf[['Brand','City', 'State/Province', 'Country','Longitude','Latitude','location']]

In [141]:
StarBdf.shape

(25600, 7)

### Lo pasamos a mongodb limpio

In [116]:
StarBdf.to_json("StarBucks.json", orient="records")

# Descargamos y limpiamos Aeropuertos

In [120]:
Airportsdf = pd.read_csv("airports-extended.csv")

Airportsdf=Airportsdf.drop(columns=['1','OurAirports'])
Airportsdf

,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081689834590001,145.391998291,5282,10,U,Pacific/Port_Moresby,airport
0,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport
1,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport
2,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport
3,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport
4,Wewak International Airport,Wewak,Papua New Guinea,WWK,AYWK,-3.583830,143.669006,19,10,U,Pacific/Port_Moresby,airport
...,...,...,...,...,...,...,...,...,...,...,...,...
10662,Kandi,Kandi,Benin,KDC,DBBK,11.145056,2.940444,958,1,U,\N,unknown
10663,Luxmore Hut Helipad,Te Anau,New Zealand,\N,\N,-45.385240,167.619250,3446,13,Z,\N,unknown
10664,Tasman Glacier,Mt. Cook,New Zealand,\N,\N,-43.569905,170.235678,4070,12,Z,\N,unknown
10665,Sokode,Sokode,Togo,\N,DXSK,8.994444,1.152778,1257,0,U,\N,unknown


In [124]:
cols={'Goroka Airport':'Name',
      'Goroka':'City',
      'Papua New Guinea':'Country',
      'GKA':'Code',
      '-6.081689834590001':'Latitude',
      '145.391998291':'Longitude',
      'airport':'Type'
    
}
Airportsdf = Airportsdf.rename(columns=cols)
Airportsdf = Airportsdf[['Name','City','Country','Code','Latitude','Longitude','Type']]

In [125]:
Airportsdf["location"] = Airportsdf[["Latitude","Longitude"]].apply(lambda x:asGeoJSON(x.Latitude,x.Longitude), axis=1)

In [127]:
Airportsdf.to_json("Airports.json", orient="records")

## Limpiamos la API de restaurantes veganos en Zurich

In [158]:
import json, requests
import os
from dotenv import load_dotenv
load_dotenv()
url = 'https://api.foursquare.com/v2/venues/explore'
clientid = os.getenv("CLIENT_ID")
clientsec = os.getenv('CLIENT_SECRET')
params = dict(
  client_id=clientid,
  client_secret=clientsec,
  v='20180323',
  near='Zurich',
  categoryId='4bf58dd8d48988d1d3941735'
)


In [159]:
resp = requests.get(url=url, params=params)
data = json.loads(resp.text)
data['response'].keys()

dict_keys(['suggestedFilters', 'geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'query', 'totalResults', 'suggestedBounds', 'groups'])

In [160]:
data['response']['groups'][0].keys()

dict_keys(['type', 'name', 'items'])

In [161]:
vegan = pd.DataFrame(data['response']['groups'][0]['items'])

In [162]:
veganRes = vegan[["venue"]].apply(lambda r: r.venue ,result_type="expand", axis=1)
veganRes = veganRes[['name','location']]

In [163]:
veganLoc = veganRes[["location"]].apply(lambda r: r.location ,result_type="expand", axis=1)

In [164]:
Veg = pd.concat([veganRes,veganLoc],axis=1)

In [165]:
Veg = Veg[['name','address','lat','lng']]

In [166]:
Veg["location"] = Veg[["lat","lng"]].apply(lambda x:asGeoJSON(x.lat,x.lng), axis=1)

In [167]:
Veg.to_json("VeganRestaurants.json", orient="records")